## Modeling


Let's start by trying to make a model to determine if a flight will be cancelled or not. To try to amplify the signal, we'll focus on just the flights that were cancelled due to weather, and flights that occured in poor weather.

In [1]:
# Import our data from the previous notebook
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

flights = pd.read_csv('flights_with_wx.csv', 
                      dtype= {'DAY_OF_WEEK': 'uint8',
                             'AIRLINE': 'category',
                             'ORIGIN_AIRPORT': 'category',
                             'DESTINATION_AIRPORT': 'category',
                             'DEPARTURE_DELAY': 'float32',
                             'ARRIVAL_DELAY': 'float32',
                             'DIVERTED': 'uint8',
                             'CANCELLED': 'uint8',
                             'CANCELLATION_REASON': 'category',
                             'ORIGIN_CEILING': 'uint16', 
                             'ORIGIN_VISIBILITY': 'float16', 
                             'ORIGIN_WIND_SPEED': 'float16',
                             'ORIGIN_PRECIPITATION': 'float32', 
                             'DESTINATION_CEILING': 'uint16', 
                             'DESTINATION_VISIBILITY': 'float16',
                             'DESTINATION_WIND_SPEED': 'float16', 
                             'DESTINATION_PRECIPITATION': 'float32'}, 
                      parse_dates=['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL'])
flights.drop('Unnamed: 0', axis=1, inplace=True)
flights_with_wxcx = flights[(flights['CANCELLATION_REASON'] != 'A') & (flights['CANCELLATION_REASON'] != 'C') & (flights['CANCELLATION_REASON'] != 'D')]
flights_with_wxcx.drop('CANCELLATION_REASON', axis=1, inplace=True)

flights_with_wxcx['ORIGIN_VISIBILITY'].fillna(10, inplace=True)
flights_with_wxcx['ORIGIN_WIND_SPEED'].fillna(7, inplace=True)
flights_with_wxcx['ORIGIN_PRECIPITATION'].fillna(0, inplace=True)
flights_with_wxcx['DESTINATION_VISIBILITY'].fillna(10, inplace=True)
flights_with_wxcx['DESTINATION_WIND_SPEED'].fillna(7, inplace=True)
flights_with_wxcx['DESTINATION_PRECIPITATION'].fillna(0, inplace=True)
flights_with_wxcx.loc[flights_with_wxcx['ORIGIN_CEILING'] > 25000, 'ORIGIN_CEILING'] = 25000
flights_with_wxcx.loc[flights_with_wxcx['DESTINATION_CEILING'] > 25000, 'DESTINATION_CEILING'] = 25000

flights_with_wxcx = flights_with_wxcx[
    (flights_with_wxcx['ORIGIN_CEILING'] < 5000) |
    (flights_with_wxcx['ORIGIN_VISIBILITY'] < 5) |
    (flights_with_wxcx['ORIGIN_WIND_SPEED'] > 20) |
    (flights_with_wxcx['DESTINATION_CEILING'] < 5000) |
    (flights_with_wxcx['DESTINATION_VISIBILITY'] < 5) |
    (flights_with_wxcx['DESTINATION_WIND_SPEED'] > 20)].dropna(
    subset=['DESTINATION_CEILING', 'DESTINATION_VISIBILITY', 'DESTINATION_WIND_SPEED'])
#-----------------------------------------------------------------------------------------

We'll start by splitting the data into a training subset and a testing subset.

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(flights_with_wxcx[['ORIGIN_CEILING', 
                                                                       'ORIGIN_VISIBILITY',
                                                                       'ORIGIN_PRECIPITATION',
                                                                       'DESTINATION_CEILING',
                                                                       'DESTINATION_VISIBILITY',
                                                                       'DESTINATION_PRECIPITATION']],
                                                    flights_with_wxcx['CANCELLED'], test_size=0.33, random_state=42)

Let's start with a basic logistic classification model and see how it does.

In [27]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression().fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    677373
           1       0.00      0.00      0.00     10597

   micro avg       0.98      0.98      0.98    687970
   macro avg       0.49      0.50      0.50    687970
weighted avg       0.97      0.98      0.98    687970



As we expected, that was pretty poor. Random Forests might do a little better with our weak signal.

In [28]:
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier(n_estimators=1500, n_jobs=-1, verbose=0).fit(X_train, y_train)
y_pred_rfc = rfc_model.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_rfc))
print(confusion_matrix(y_test, y_pred_rfc))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    677373
           1       0.37      0.09      0.15     10597

   micro avg       0.98      0.98      0.98    687970
   macro avg       0.68      0.54      0.57    687970
weighted avg       0.98      0.98      0.98    687970

[[675711   1662]
 [  9631    966]]


Better, but still not great. Let's add a few more columns of data and see if that helps. We'll need to one-hot encode our catagorical variables first.

In [3]:
flights_encoded = pd.concat([
    pd.get_dummies(flights_with_wxcx[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']]), 
    flights_with_wxcx.drop(['SCHEDULED_DEPARTURE', 'AIRLINE', 'DAY_OF_WEEK', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_ARRIVAL', 'DEPARTURE_DELAY', 'DIVERTED', 'ARRIVAL_DELAY'], axis=1)],
    axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(flights_encoded.drop('CANCELLED', axis=1),
                                                    flights_encoded['CANCELLED'], test_size=0.33, random_state=42)

In [31]:
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier(n_estimators=100, n_jobs=-1, verbose=0).fit(X_train, y_train)
y_pred_rfc = rfc_model.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_rfc))
print(confusion_matrix(y_test, y_pred_rfc))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    677373
           1       0.84      0.11      0.19     10597

   micro avg       0.99      0.99      0.99    687970
   macro avg       0.91      0.55      0.59    687970
weighted avg       0.98      0.99      0.98    687970

[[677153    220]
 [  9448   1149]]


This actually isn't bad! To explain these results in plain english: only 11% of the flights that were actually cancelled were predicted to be cancelled by this Random Forest model. However, 84% of the flights that the model predicted to be cancelled were actually cancelled. If this model were to be used operationally, you would not rely on it as the sole source for predicting whether or not a flight would be cancelled, since it fails to catch 89% of flights that are cancelled. However, whenever it does predict a particular flight to be cancelled, you could rely on it to be correct and start implementing contingency solutions for getting passengers to their destination. 

We could potentially increase the precison and recall of this model with more estimators, but that would require significantly more processing power than my laptop can provide! Similarly, a boosted method like AdaBoost or Gradient Tree Boosting could produce better results, but their inherent sequential nature does not allow for parallel computation, making them time-consuming to train on a laptop.

Lastly for our cancellation modeling, let's try a Deep Neural Network with TensorFlow. We'll start by scaling our numerical feature columns.

In [16]:
data = flights_with_wxcx[['AIRLINE', 
                          'ORIGIN_AIRPORT',
                          'DESTINATION_AIRPORT',
                          'ORIGIN_CEILING', 
                          'ORIGIN_VISIBILITY', 
                          'ORIGIN_WIND_SPEED',
                          'DESTINATION_CEILING',
                          'DESTINATION_VISIBILITY',
                          'DESTINATION_WIND_SPEED',
                          'CANCELLED']]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(data.drop(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'CANCELLED'], axis=1))
scaled_features = scaler.fit_transform(data.drop(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'CANCELLED'], axis=1))
df_feat = pd.concat([pd.DataFrame(scaled_features,columns=data.columns[3:-1]), 
                     flights_with_wxcx['AIRLINE'].reset_index(drop=True).astype(str),
                     flights_with_wxcx['ORIGIN_AIRPORT'].reset_index(drop=True).astype(str),
                     flights_with_wxcx['DESTINATION_AIRPORT'].reset_index(drop=True).astype(str)], 
                     axis=1)

Next, we'll build our feature columns.

In [24]:
import os
import tensorflow as tf
tf.logging.set_verbosity('FATAL')

origin_ceiling = tf.feature_column.numeric_column("ORIGIN_CEILING")
origin_vis = tf.feature_column.numeric_column('ORIGIN_VISIBILITY')
origin_wind = tf.feature_column.numeric_column('ORIGIN_WIND_SPEED')
dest_ceiling =tf.feature_column.numeric_column('DESTINATION_CEILING')
dest_vis = tf.feature_column.numeric_column('DESTINATION_VISIBILITY')
dest_wind = tf.feature_column.numeric_column('DESTINATION_WIND_SPEED')
airline = tf.feature_column.categorical_column_with_vocabulary_list(key='AIRLINE', vocabulary_list=flights_with_wxcx['AIRLINE'].unique().astype(str).tolist())
origin_airport = tf.feature_column.categorical_column_with_vocabulary_list(key='ORIGIN_AIRPORT', vocabulary_list=flights_with_wxcx['ORIGIN_AIRPORT'].unique().astype(str).tolist())
dest_airport = tf.feature_column.categorical_column_with_vocabulary_list(key='DESTINATION_AIRPORT', vocabulary_list=flights_with_wxcx['DESTINATION_AIRPORT'].unique().astype(str).tolist())

feat_cols = [origin_ceiling, 
             origin_vis, 
             origin_wind, 
             dest_ceiling, 
             dest_vis, 
             dest_wind, 
             tf.feature_column.indicator_column(airline), 
             tf.feature_column.indicator_column(origin_airport), 
             tf.feature_column.indicator_column(dest_airport)]

Finally, we'll do another train_test_split with our selected columns, create the model, train it, and check its performance.

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_feat, data['CANCELLED'].reset_index(drop=True), test_size=0.33, random_state=42, shuffle=False)


dnn_classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10], n_classes=2,feature_columns=feat_cols)
# We use an unsually large batch size here. Because the percent of cancelled flights is so low, we want to 
# try to ensure each batch contains at least one cancelled flight to avoid underfitting.
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100, shuffle=True)
dnn_classifier.train(input_fn=input_func, steps=100000)

pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)
dnn_flight_predictions = list(dnn_classifier.predict(input_fn=pred_fn))
final_preds  = []
for pred in dnn_flight_predictions:
    final_preds.append(pred['class_ids'][0])
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,final_preds))
print(classification_report(y_test,final_preds))

[[681747      0]
 [  6223      0]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    681747
           1       0.00      0.00      0.00      6223

   micro avg       0.99      0.99      0.99    687970
   macro avg       0.50      0.50      0.50    687970
weighted avg       0.98      0.99      0.99    687970



We could try to play around with the number of hidden layers and the number of neurons per layer, but we'll stick with the Random Forest model and see if we can improve it with more training.

Let's try XGBoost and compare it to our Random Forest Model

In [20]:
import xgboost
xgb_model = xgboost.XGBClassifier(max_depth=6, n_estimators=100, scale_pos_weight=(1/y_train.mean()), n_jobs=4).fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       1.00      0.84      0.91    677373
           1       0.07      0.76      0.12     10597

   micro avg       0.84      0.84      0.84    687970
   macro avg       0.53      0.80      0.52    687970
weighted avg       0.98      0.84      0.90    687970

[[566486 110887]
 [  2564   8033]]


In [22]:
import xgboost
xgb_model = xgboost.XGBClassifier(max_depth=6, n_estimators=150, n_jobs=4, scale_pos_weight=10).fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99    677373
           1       0.27      0.40      0.32     10597

   micro avg       0.97      0.97      0.97    687970
   macro avg       0.63      0.69      0.65    687970
weighted avg       0.98      0.97      0.98    687970

[[665933  11440]
 [  6387   4210]]


In [24]:
import xgboost
xgb_model = xgboost.XGBClassifier(max_depth=10, n_estimators=150, n_jobs=4, scale_pos_weight=10).fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    677373
           1       0.34      0.44      0.38     10597

   micro avg       0.98      0.98      0.98    687970
   macro avg       0.67      0.71      0.69    687970
weighted avg       0.98      0.98      0.98    687970

[[668442   8931]
 [  5969   4628]]


In [26]:
import xgboost
xgb_model = xgboost.XGBClassifier(max_depth=10, n_estimators=400, n_jobs=4, scale_pos_weight=2).fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    677373
           1       0.67      0.26      0.37     10597

   micro avg       0.99      0.99      0.99    687970
   macro avg       0.83      0.63      0.68    687970
weighted avg       0.98      0.99      0.98    687970

[[676039   1334]
 [  7850   2747]]


In [27]:
import xgboost
xgb_model = xgboost.XGBClassifier(max_depth=15, n_estimators=400, n_jobs=4, scale_pos_weight=2).fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    677373
           1       0.70      0.28      0.40     10597

   micro avg       0.99      0.99      0.99    687970
   macro avg       0.84      0.64      0.70    687970
weighted avg       0.98      0.99      0.98    687970

[[676067   1306]
 [  7610   2987]]
